In [12]:
import sys
sys.path.append("../")

import gcsfs
import zarr

import dask.array as da

from src.cmesrc.config import CMESRC_DB
import sqlite3
from matplotlib.colors import Normalize

import matplotlib.pyplot as plt
from src.cmesrc.utils import filepaths_updated_swan_data, read_SWAN_filepath
import pandas as pd

In [6]:
con = sqlite3.connect(CMESRC_DB)
con.execute("PRAGMA foreign_keys = ON")
cur = con.cursor()


In [22]:
SWAN = filepaths_updated_swan_data()

df = read_SWAN_filepath(SWAN[5342])
df


==CACHING SWAN DATA.==



100%|██████████| 4098/4098 [00:00<00:00, 1186460.81it/s]

,Timestamp,TOTUSJH,TOTBSQ,TOTPOT,TOTUSJZ,ABSNJZH,SAVNCPP,USFLUX,TOTFZ,MEANPOT,EPSZ,MEANSHR,SHRGT45,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZH,TOTFY,MEANJZD,MEANALP,TOTFX,EPSY,EPSX,R_VALUE,CRVAL1,CRLN_OBS,CRLT_OBS,CRVAL2,HC_ANGLE,SPEI,LAT_MIN,LON_MIN,LAT_MAX,LON_MAX,QUALITY,BFLARE,BFLARE_LABEL,CFLARE,CFLARE_LABEL,MFLARE,MFLARE_LABEL,XFLARE,XFLARE_LABEL,BFLARE_LOC,BFLARE_LABEL_LOC,CFLARE_LOC,CFLARE_LABEL_LOC,MFLARE_LOC,MFLARE_LABEL_LOC,XFLARE_LOC,XFLARE_LABEL_LOC,XR_MAX,XR_QUAL,IS_TMFI,LONDTMIN,LONDTMAX,LATDTMIN,LATDTMAX,IRBB
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-03-16 14:12:00.000,2015-03-16 14:12:00.000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,-0.000000e+00,NaN,NaN,0.000000,56.955975,149.586975,-7.134594,21.694901,92.304722,False,18.259436,-85.079811,19.345810,-82.883278,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,5.210500e-07,12,False,-106.360001,-78.902000,14.7086,28.6812,False
2015-03-16 14:24:00.000,2015-03-16 14:24:00.000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,-0.000000e+00,NaN,NaN,0.000000,56.957035,149.477036,-7.134726,21.694901,92.207500,False,18.059372,-85.525635,19.644432,-82.376305,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,5.012400e-07,12,False,-106.249001,-78.791000,14.7086,28.6812,False
2015-03-16 14:36:00.000,2015-03-16 14:36:00.000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,-0.000000e+00,NaN,NaN,0.000000,56.958065,149.367065,-7.134888,21.694901,92.110274,False,17.825485,-85.966080,20.095608,-81.748520,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,4.906500e-07,12,False,-106.138000,-78.680000,14.7086,28.6812,False
2015-03-16 14:48:00.000,2015-03-16 14:48:00.000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,-0.000000e+00,NaN,NaN,0.000000,56.959049,149.257050,-7.135080,21.694901,92.013048,False,17.636173,-86.439796,20.213789,-81.246529,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,4.237200e-07,12,False,-106.027000,-78.569000,14.7086,28.6812,False
2015-03-16 15:00:00.000,2015-03-16 15:00:00.000,0.092999,8.517374e+05,5.066071e+18,3.603382e+09,0.044601,6.030590e+09,3.609717e+18,-8.384312e+20,381.468598,-0.741224,19.296270,0.000000,23.444846,15.039033,17.400189,21.504321,0.004460,2.141416e+20,1.158975,0.060147,2.576869e+20,-0.189314,-0.227811,3.345845,56.960018,149.147018,-7.135300,21.694901,91.915819,False,17.584736,-87.221977,20.269342,-80.950218,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,4.045200e-07,12,False,-105.916000,-78.458000,14.7086,28.6812,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-03-29 23:12:00.000,2015-03-29 23:12:00.000,0.602432,1.358405e+07,6.347331e+19,2.096055e+10,0.185346,1.995185e+10,4.558275e+19,-1.530717e+22,473.213529,-0.848502,17.732695,6.930693,23.384571,51.398492,65.805380,18.647529,0.001835,1.444006e+21,0.376778,0.014763,2.231628e+21,-0.080044,-0.123703,2.753484,418.751160,333.215515,-6.684560,21.694901,86.073075,False,15.126799,78.714775,22.740644,87.905861,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,7.799000e-07,12,False,71.806702,99.264603,14.7086,28.6812,False
2015-03-29 23:24:00.000,2015-03-29 23:24:00.000,0.319865,3.687637e+06,1.118075e+20,1.458579e+10,0.020430,1.186478e+10,1.418353e+19,-3.431154e+21,1830.208380,-0.700615,48

In [11]:
cur.execute("PRAGMA table_info(harps_bbox)")

[(0, 'harpnum', 'INTEGER', 0, None, 1), (1, 'timestamp', 'TEXT', 0, None, 2), (2, 'LONDTMIN', 'REAL', 0, None, 0), (3, 'LONDTMAX', 'REAL', 0, None, 0), (4, 'LATDTMIN', 'REAL', 0, None, 0), (5, 'LATDTMAX', 'REAL', 0, None, 0), (6, 'IRBB', 'INTEGER', 0, None, 0)]


In [20]:
new_df = pd.read_sql_query("SELECT timestamp, londtmin, londtmax, latdtmin, latdtmax FROM harps_bbox WHERE harpnum = 5342", con)
new_df.set_index("timestamp", inplace=True)
new_df

,LONDTMIN,LONDTMAX,LATDTMIN,LATDTMAX
timestamp,,,,
2015-03-16 14:12:00,-106.360001,-78.902000,14.7086,28.6812
2015-03-16 14:24:00,-106.249001,-78.791000,14.7086,28.6812
2015-03-16 14:36:00,-106.138000,-78.680000,14.7086,28.6812
2015-03-16 14:48:00,-106.027000,-78.569000,14.7086,28.6812
2015-03-16 15:00:00,-105.916000,-78.458000,14.7086,28.6812
...,...,...,...,...
2015-03-29 23:12:00,71.806702,99.264603,14.7086,28.6812
2015-03-29 23:24:00,71.917702,99.375603,14.7086,28.6812
2015-03-29 23:36:00,72.028702,99.486603,14.7086,28.6812


In [21]:
# Merge based on index values

merged = df.merge(new_df, how="inner", left_index=True, right_index=True)
merged

,Timestamp,TOTUSJH,TOTBSQ,TOTPOT,TOTUSJZ,ABSNJZH,SAVNCPP,USFLUX,TOTFZ,MEANPOT,EPSZ,MEANSHR,SHRGT45,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZH,TOTFY,MEANJZD,MEANALP,TOTFX,EPSY,EPSX,R_VALUE,CRVAL1,CRLN_OBS,CRLT_OBS,CRVAL2,HC_ANGLE,SPEI,LAT_MIN,LON_MIN,LAT_MAX,LON_MAX,QUALITY,BFLARE,BFLARE_LABEL,CFLARE,CFLARE_LABEL,MFLARE,MFLARE_LABEL,XFLARE,XFLARE_LABEL,BFLARE_LOC,BFLARE_LABEL_LOC,CFLARE_LOC,CFLARE_LABEL_LOC,MFLARE_LOC,MFLARE_LABEL_LOC,XFLARE_LOC,XFLARE_LABEL_LOC,XR_MAX,XR_QUAL,IS_TMFI,LONDTMIN_x,LONDTMAX_x,LATDTMIN_x,LATDTMAX_x,IRBB,LONDTMIN_y,LONDTMAX_y,LATDTMIN_y,LATDTMAX_y
2015-03-16 14:12:00.000,2015-03-16 14:12:00.000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,-0.000000e+00,NaN,NaN,0.000000,56.955975,149.586975,-7.134594,21.694901,92.304722,False,18.259436,-85.079811,19.345810,-82.883278,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,5.210500e-07,12,False,-106.360001,-78.902000,14.7086,28.6812,False,-106.360001,-78.902000,14.7086,28.6812
2015-03-16 14:24:00.000,2015-03-16 14:24:00.000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,-0.000000e+00,NaN,NaN,0.000000,56.957035,149.477036,-7.134726,21.694901,92.207500,False,18.059372,-85.525635,19.644432,-82.376305,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,5.012400e-07,12,False,-106.249001,-78.791000,14.7086,28.6812,False,-106.249001,-78.791000,14.7086,28.6812
2015-03-16 14:36:00.000,2015-03-16 14:36:00.000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,-0.000000e+00,NaN,NaN,0.000000,56.958065,149.367065,-7.134888,21.694901,92.110274,False,17.825485,-85.966080,20.095608,-81.748520,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,4.906500e-07,12,False,-106.138000,-78.680000,14.7086,28.6812,False,-106.138000,-78.680000,14.7086,28.6812
2015-03-16 14:48:00.000,2015-03-16 14:48:00.000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,-0.000000e+00,NaN,NaN,0.000000,56.959049,149.257050,-7.135080,21.694901,92.013048,False,17.636173,-86.439796,20.213789,-81.246529,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,4.237200e-07,12,False,-106.027000,-78.569000,14.7086,28.6812,False,-106.027000,-78.569000,14.7086,28.6812
2015-03-16 15:00:00.000,2015-03-16 15:00:00.000,0.092999,8.517374e+05,5.066071e+18,3.603382e+09,0.044601,6.030590e+09,3.609717e+18,-8.384312e+20,381.468598,-0.741224,19.296270,0.000000,23.444846,15.039033,17.400189,21.504321,0.004460,2.141416e+20,1.158975,0.060147,2.576869e+20,-0.189314,-0.227811,3.345845,56.960018,149.147018,-7.135300,21.694901,91.915819,False,17.584736,-87.221977,20.269342,-80.950218,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,4.045200e-07,12,False,-105.916000,-78.458000,14.7086,28.6812,False,-105.916000,-78.458000,14.7086,28.6812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-03-29 23:12:00.000,2015-03-29 23:12:00.000,0.602432,1.358405e+07,6.347331e+19,2.096055e+10,0.185346,1.995185e+10,4.558275e+19,-1.530717e+22,473.213529,-0.848502,17.732695,6.930693,23.384571,51.398492,65.805380,18.647529,0.001835,1.444006e+21,0.376778,0.014763,2.231628e+21,-0.080044,-0.123703,2.753484,418.751160,333.215515,-6.684560,21.694901,86.073075,False,15.126799,78.714775,22.740644,87.905861,0.0,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,0.0,None,7.799000e-07,12,False,71.806702,99.26460